In [ ]:
import math
import random

# このクラスはモンテカルロ木探索（MCTS）のノードを表します。
# ゲームの状態、その親ノード、子ノード、訪問回数、および総価値を保存します。
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0

    def is_fully_expanded(self):
        # 現在の状態から可能なすべてのアクションが子ノードとして展開されているかを確認します。
        return len(self.children) == len(self.state.get_legal_actions())

    def best_child(self, exploration_weight=1.):
        # 木の上部信頼限界（UCT）式に基づいて、探索と利用のバランスを取りながら最良の子ノードを選択します。
        if not self.children:
            return None
        return max(self.children, key=lambda child: child.value / (child.visits + 1) +
                   exploration_weight * math.sqrt(math.log(self.visits + 1) / (child.visits + 1)))

    def expand(self):
        # 未探索のアクションに対して、新しい子ノードを作成して現在のノードを展開します。
        tried_actions = [child.state.last_action for child in self.children]
        legal_actions = self.state.get_legal_actions()
        for action in legal_actions:
            if action not in tried_actions:
                next_state = self.state.move(action)
                child_node = Node(next_state, parent=self)
                self.children.append(child_node)
                return child_node

    def backpropagate(self, result):
        # シミュレーションの結果をノードの価値と訪問回数に反映し、再帰的に親ノードにも結果を伝播します。
        self.visits += 1
        self.value += result
        if self.parent:
            self.parent.backpropagate(-result)

# このクラスは三目並べ（Tic Tac Toe）ゲームを表します。
# 盤面の状態、現在のプレイヤー、ゲームのロジックを含みます。
class TicTacToe:
    def __init__(self):
        # 空の状態で三目並べの盤面を初期化し、現在のプレイヤーを 'X' に設定します。
        self.board = [' '] * 9
        self.current_player = 'X'
        self.last_action = None

    def move(self, index):
        # 指定された位置に移動した後の新しい三目並べの状態を返します。
        new_state = TicTacToe()
        new_state.board = self.board[:]
        new_state.board[index] = self.current_player
        new_state.current_player = 'O' if self.current_player == 'X' else 'X'
        new_state.last_action = index
        return new_state

    def get_legal_actions(self):
        # 現在の盤面で可能な合法な移動のインデックスのリストを返します。
        return [i for i in range(9) if self.board[i] == ' ']

    def is_terminal(self):
        # ゲームが終了状態（勝利または引き分け）に達したかどうかを確認します。
        return self.get_winner() is not None or ' ' not in self.board

    def get_winner(self):
        # 勝者（'X' または 'O'）がいる場合はそれを返し、いない場合は None を返します。
        for (i, j, k) in [(0, 1, 2), (3, 4, 5), (6, 7, 8), (0, 3, 6), (1, 4, 7), (2, 5, 8), (0, 4, 8), (2, 4, 6)]:
            if self.board[i] == self.board[j] == self.board[k] and self.board[i] != ' ':
                return self.board[i]
        return None

    def __str__(self):
        # 現在の盤面の状態を文字列として返し、コンソールに表示します。
        return "\n".join([" | ".join(self.board[i:i + 3]) for i in range(0, 9, 3)])

# 与えられた初期状態から最良の手を決定するためにモンテカルロ木探索（MCTS）を実行します。
def mcts(initial_state, itermax=1000, exploration_weight=1.):
    root = Node(initial_state)

    for _ in range(itermax):
        node = root
        while not node.state.is_terminal() and node.is_fully_expanded():
            node = node.best_child(exploration_weight)

        if not node.state.is_terminal():
            node = node.expand()

        result = 1 if node.state.get_winner() == initial_state.current_player else -1 if node.state.get_winner() else 0
        node.backpropagate(result)

    return root.best_child(0).state.last_action

if __name__ == '__main__':
    # 三目並べのゲームをプレイするメイン関数です。'X' は MCTS を使用し、'O' は人間がプレイします。
    state = TicTacToe()
    while not state.is_terminal():
        if state.current_player == 'X':
            action = mcts(state, itermax=1000)
        else:
            while True:
                try:
                    action = int(input("あなたの手を入力してください (0-8): "))
                    if action in state.get_legal_actions():
                        break
                    else:
                        print("無効な手です。もう一度試してください。")
                except ValueError:
                    print("0から8の数字を入力してください。")
        state = state.move(action)
        print(state)
        print("-----")

    winner = state.get_winner()
    if winner:
        print(f"勝者: {winner}")
    else:
        print("引き分けです！")

X |   |  
  |   |  
  |   |  
-----
あなたの手を入力してください (0-8): 4
X |   |  
  | O |  
  |   |  
-----
X |   |  
X | O |  
  |   |  
-----
あなたの手を入力してください (0-8): 6
X |   |  
X | O |  
O |   |  
-----
X | X |  
X | O |  
O |   |  
-----
あなたの手を入力してください (0-8): 2
X | X | O
X | O |  
O |   |  
-----
勝者: O
